In [0]:
!git clone https://github.com/christiansafka/img2vec.git
!git clone https://github.com/nlpaueb/bio_image_caption.git

In [0]:
import bio_image_caption.SiVL19.create_vocabulary as vocab
import bio_image_caption.SiVL19.frequency_baseline as freq
import bio_image_caption.SiVL19.onenn_baseline as onenn
import bio_image_caption.SiVL19.wmd_evaluation as wmd
import bio_image_caption.SiVL19.coco_evaluation as coco_eval
import os
import pandas as pd

In [0]:
# Create the dataset folder, download data and split to train and test
!python bio_image_caption/SiVL19/get_iu_xray.py

In [0]:
# Define the dataset folder and train and test files
dataset_path = "iu_xray/"
train_path = os.path.join(dataset_path, "train_images.tsv")
test_path = os.path.join(dataset_path, "test_images.tsv")

# Define the folder in which to save the results
!mkdir "iu_xray_results"
results_path = "iu_xray_results"

# Define the sub folder that contains the images
images_path = os.path.join(dataset_path, "iu_xray_images")

In [0]:
# Get statistics and create vocabulary for the dataset
print("Processing IU X-ray:")
mean_length = vocab.create_vocabulary(train_path, results_path)

In [0]:
# Run the frequency baseline for the dataset
print("Frequency baseline:")
freq_results = freq.most_frequent(train_path, test_path, results_path, mean_length)

In [0]:
# Run the onenn baseline for the dataset

# If a GPU is available set cuda to True to run the 1NN baseline
cuda = True
	
print("1NN baseline:")
onenn_results = onenn.most_similar(train_path, test_path, images_path, results_path, cuda)

In [0]:
# Download pre-trained word embeddings
!wget https://archive.org/download/pubmed2018_w2v_200D.tar/pubmed2018_w2v_200D.tar.gz
# Unzip word embeddings
!tar xvzf pubmed2018_w2v_200D.tar.gz

In [0]:
# Evaluation

# Load gold test image_id-caption pairs to a dictionary
test_data = pd.read_csv(test_path, sep="\t", header=None, names=["image_ids", "captions"])
gold = dict(zip(test_data.image_ids, test_data.captions))

# Define pretrained embeddings path
embeddings_path = "pubmed2018_w2v_200D/pubmed2018_w2v_200D.bin"

# WMD evaluation
print("WMD evaluation for frequency baseline:")
wmd.compute_wmd(gold, freq_results, embeddings_path)
print("WMD evaluation for onenn baseline:")
wmd.compute_wmd(gold, onenn_results, embeddings_path)

# MSCOCO evaluation
print("MSCOCO evaluation for frequency baseline:")
coco_eval.compute_scores(gold, freq_results)
print("MSCOCO evaluation for onenn baseline:")
coco_eval.compute_scores(gold, onenn_results)